# NullClass Internship: Real-Time Google Play Store Data Analytics – Python
Combined Tasks 1–5 in a single script for GitHub submission.

1. Create a single .py script that bundles Tasks 1–5 with all filters and time windows.
2. The script expects a file named 'googleplaystore.csv' in the same folder.
3. It uses Plotly for charts and handles common data-cleaning quirks in the Google Play dataset.

# Import Laibrary

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import pytz

# Load data

In [2]:
df = pd.read_csv(r'C:\Users\vishal\Desktop\Dataset\Play Store Data.csv')
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB
None


# Clean Data

In [4]:
# Drop missing important values
df = df.dropna(subset=['App', 'Category', 'Installs'])

In [5]:
# Clean Installs
df['Installs'] = df['Installs'].str.replace('[+,]', '', regex=True)
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')

In [6]:
# Clean Size → MB
def size_to_mb(x):
    if isinstance(x, str):
        if 'M' in x:
            return float(x.replace('M', ''))
        elif 'k' in x:
            return float(x.replace('k', ''))/1024
        elif 'G' in x:
            return float(x.replace('G', ''))*1024
    return None

df['Size_MB'] = df['Size'].apply(size_to_mb)

In [7]:
# Clean Reviews
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

In [8]:
# Parse Last Updated
df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')

In [9]:
# Android Version
import re
df['Android_Ver_Num'] = pd.to_numeric(df['Android Ver'].str.extract(r'(\d+\.\d+)')[0], errors='coerce')

In [10]:
# Add dummy revenue if missing
if 'Revenue' not in df.columns:
    df['Revenue'] = df['Installs'] * 0.05

In [11]:
# Add Country column if missing (for Task 2)
if 'Country' not in df.columns:
    import numpy as np
    countries = ['US','IN','UK','DE','FR','BR','CA','AU']
    df['Country'] = np.random.choice(countries, len(df))

# Time Utility (IST)

In [12]:
def is_visible_time(start_h, end_h):
    ist = pytz.timezone('Asia/Kolkata')
    now = datetime.now(ist)
    return start_h <= now.hour < end_h

# Task 1
1. Grouped bar: avg rating & total reviews for top 10 categories by installs.
2. Filters: drop categories where avg rating < 4.0, size < 10MB, last update should be January.
3. Time window: 3–5 PM IST

In [13]:
if is_visible_time(15, 17):
    jan_filter = df[(df['Rating'] >= 4.0) & (df['Size_MB'] >= 10) & (df['Last Updated'].dt.month == 1)]
    grouped = jan_filter.groupby('Category').agg({
        'Rating': 'mean',
        'Reviews': 'sum'
    }).reset_index().nlargest(10, 'Reviews')

    fig = px.bar(grouped, x='Category', y=['Rating','Reviews'], barmode='group',
                 title='Top 10 Categories (Jan, Rating>=4.0, Size>=10MB)')
    fig.show()
else:
    print("Task 1: Chart visible only between 3–5 PM IST")


# Task 2
1. horopleth: global installs by Category (top 5 categories), highlight categories where installs > 1,000,000, exclude categories starting with A/C/G/S.
2. Time window: 6–8 PM IST

In [14]:
if is_visible_time(18, 20):
    filtered = df[~df['Category'].str.startswith(('A','C','G','S'))]
    grouped = filtered.groupby(['Country','Category'])['Installs'].sum().reset_index()
    top5 = grouped.groupby('Category')['Installs'].sum().nlargest(5).index
    grouped = grouped[grouped['Category'].isin(top5)]
    grouped['Highlight'] = grouped['Installs'] > 1_000_000

    fig = px.choropleth(grouped, locations='Country', locationmode='country names',
                        color='Installs', hover_name='Category',
                        title='Global Installs (Top 5 Categories, >1M Highlight)')
    fig.show()
else:
    print("Task 2: Chart visible only between 6–8 PM IST")

Task 2: Chart visible only between 6–8 PM IST


# Task 3
1. Dual-axis chart comparing average installs and revenue for Free vs Paid
2. within top 3 categories. Filters: Installs>=10k, Revenue>=10k, Android>4.0, Size>15MB, Content Rating='Everyone', App name length<=30.
3. Time window: 1–2 PM IST

In [15]:
if is_visible_time(13, 14):
    filt = df[(df['Installs'] >= 10000) & (df['Revenue'] >= 10000) &
              (df['Android_Ver_Num'] > 4.0) & (df['Size_MB'] > 15) &
              (df['Content Rating'] == 'Everyone') & (df['App'].str.len() <= 30)]
    top3 = filt.groupby('Category')['Installs'].sum().nlargest(3).index
    filt = filt[filt['Category'].isin(top3)]
    grouped = filt.groupby(['Category','Type']).agg({'Installs':'mean','Revenue':'mean'}).reset_index()

    fig = px.bar(grouped, x='Category', y='Installs', color='Type', barmode='group')
    fig.add_scatter(x=grouped['Category'], y=grouped['Revenue'], mode='lines+markers', name='Revenue')
    fig.show()
else:
    print("Task 3: Chart visible only between 1–2 PM IST")

Task 3: Chart visible only between 1–2 PM IST


# Task 4
1. Time series: total installs over time, segmented by category.
2. Highlight periods of >20% MoM increase (markers under curve).
3. Filters:
 - App name not starting with x/y/z
 - Category starts with E/C/B
 - Translate: Beauty->Hindi (सौंदर्य), Business->Tamil (வணிகம்), Dating->German (Partnersuche)
 - Reviews > 500
 - App name must NOT contain letter 'S'.
   
4. Time window: 6–9 PM IST

In [16]:
if is_visible_time(18, 21):
    filt = df[(~df['App'].str.startswith(('x','y','z')))
              & (df['Category'].str.startswith(('E','C','B')))
              & (df['Reviews'] > 500)
              & (~df['App'].str.contains('S'))]
    filt['Month'] = filt['Last Updated'].dt.to_period('M')
    grouped = filt.groupby(['Month','Category'])['Installs'].sum().reset_index()
    grouped['Month'] = grouped['Month'].astype(str)
    grouped['Prev'] = grouped.groupby('Category')['Installs'].shift(1)
    grouped['Growth'] = (grouped['Installs'] - grouped['Prev'])/grouped['Prev']*100
    grouped['Highlight'] = grouped['Growth'] > 20

    translations = {"Beauty":"सौंदर्य","Business":"வணிகம்","Dating":"Liebe"}
    grouped['Category'] = grouped['Category'].replace(translations)

    fig = px.line(grouped, x='Month', y='Installs', color='Category',
                  title='Monthly Installs by Category (Growth>20 Highlight)')
    for cat in grouped['Category'].unique():
        highlight = grouped[(grouped['Category']==cat) & (grouped['Highlight'])]
        fig.add_scatter(x=highlight['Month'], y=highlight['Installs'],
                        mode='markers', marker=dict(size=10,color='red'),
                        name=f"{cat} Growth>20%")
    fig.show()
else:
    print("Task 4: Chart visible only between 6–9 PM IST")

Task 4: Chart visible only between 6–9 PM IST


# Task 5
1. Bubble chart: size (MB) vs average rating, bubble size = installs.
2. Filters:
 - Rating > 3.5
 - Categories in [Game, Beauty, Business, Comics, Communication, Dating, Entertainment, Social, Events]
 - Reviews > 500
 - App name must NOT contain letter 'S'
 - Sentiment subjectivity > 0.5 (if column available)
 - Highlight Game category in pink
 - Installs > 50k.

3. Time window: 5–7 PM IST

In [17]:
if is_visible_time(17, 19):
    filt = df[(df['Rating'] > 3.5)
              & (df['Category'].isin(['Comics','Communication','Beauty','Business','Dating']))
              & (df['Reviews'] > 500)
              & (~df['App'].str.contains('S'))
              & (df['Installs'] > 50000)]
    if 'Subjectivity' in df.columns:
        filt = filt[filt['Subjectivity'] > 0.5]

    translations = {"Beauty":"सौंदर्य","Business":"வணிகம்","Dating":"Liebe"}
    filt['Category'] = filt['Category'].replace(translations)

    fig = px.scatter(filt, x='Size_MB', y='Rating', size='Installs', color='Category',
                     hover_name='App', title='Bubble Chart: Size vs Rating by Category')
    game_apps = filt[filt['Category'].str.contains('Game', case=False, na=False)]
    if not game_apps.empty:
        fig.add_scatter(x=game_apps['Size_MB'], y=game_apps['Rating'], mode='markers',
                        marker=dict(size=15,color='pink'), name='Game Highlight')
    fig.show()
else:
    print("Task 5: Chart visible only between 5–7 PM IST")

Task 5: Chart visible only between 5–7 PM IST
